In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError


In [2]:
import urllib3
urllib3.disable_warnings()

In [3]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "DwOAtXZC3kMLCPadyoQY"),
    verify_certs=False  # Disable SSL certificate verification
)

# Test connection
if es.ping():
    print("Connected to Elasticsearch successfully!")
else:
    print("Failed to connect to Elasticsearch.")

Connected to Elasticsearch successfully!


In [4]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt


In [5]:
df = pd.read_csv("movies.csv").loc[:50000]


In [6]:
df.head(5)



,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,352056482.0,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,65675816.0,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,407045464.0,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
3,693134,Dune: Part Two,Science Fiction-Adventure,en,Follow the mythic journey of Paul Atreides as ...,4742.163,Legendary Pictures,2024-02-27,190000000.0,683813734.0,167.0,Released,Long live the fighters.,8.300,2770.0,Timothée Chalamet-Zendaya-Rebecca Ferguson-Jav...,epic-based on novel or book-fight-sandstorm-sa...,/czembW0Rk1Ke7lCJGahbOhdCuhV.jpg,/xOMo8BRK7PfcJv9JCnx7s5hj0PX.jpg,438631-763215-792307-1011985-467244-634492-359...
4,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,475766228.0,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...


In [7]:
df.shape

(50001, 20)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    50001 non-null  int64  
 1   title                 50001 non-null  object 
 2   genres                48812 non-null  object 
 3   original_language     50001 non-null  object 
 4   overview              49125 non-null  object 
 5   popularity            50001 non-null  float64
 6   production_companies  43926 non-null  object 
 7   release_date          49841 non-null  object 
 8   budget                50001 non-null  float64
 9   revenue               50001 non-null  float64
 10  runtime               49912 non-null  float64
 11  status                50001 non-null  object 
 12  tagline               25819 non-null  object 
 13  vote_average          50001 non-null  float64
 14  vote_count            50001 non-null  float64
 15  credits            

In [9]:
#print null values in each column
null_values = df.isnull().sum()
print(null_values)

id                          0
title                       0
genres                   1189
original_language           0
overview                  876
popularity                  0
production_companies     6075
release_date              160
budget                      0
revenue                     0
runtime                    89
status                      0
tagline                 24182
vote_average                0
vote_count                  0
credits                  1243
keywords                13579
poster_path               468
backdrop_path            4556
recommendations         23697
dtype: int64


In [10]:
#remove records where overview, title and genres is null
df = df.dropna(subset=['overview', 'genres', 'title'])
print(df.isnull().sum())

id                          0
title                       0
genres                      0
original_language           0
overview                    0
popularity                  0
production_companies     5063
release_date               80
budget                      0
revenue                     0
runtime                    16
status                      0
tagline                 22467
vote_average                0
vote_count                  0
credits                   744
keywords                12197
poster_path               264
backdrop_path            3457
recommendations         22073
dtype: int64


In [11]:
df.shape

(48155, 20)

In [12]:

duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [13]:
df = df.drop_duplicates()   

In [14]:
df.isna().value_counts()

id     title  genres  original_language  overview  popularity  production_companies  release_date  budget  revenue  runtime  status  tagline  vote_average  vote_count  credits  keywords  poster_path  backdrop_path  recommendations
False  False  False   False              False     False       False                 False         False   False    False    False   False    False         False       False    False     False        False          False              13738
                                                                                                                                                                                                                       True                6691
                                                                                                                                     True     False         False       False    False     False        False          False               6420
                                                 

In [15]:
df.fillna("None",inplace=True)

C:\Users\sajan\AppData\Local\Temp\ipykernel_25692\2303779330.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("None",inplace=True)


In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\irwa Project_2\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\irwa Project_2\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
# Create the vectors
df["OverviewVector"] = df["overview"].apply(lambda x: model.encode(x))



In [18]:
df.head()
print(df.dtypes)

id                        int64
title                    object
genres                   object
original_language        object
overview                 object
popularity              float64
production_companies     object
release_date             object
budget                  float64
revenue                 float64
runtime                  object
status                   object
tagline                  object
vote_average            float64
vote_count              float64
credits                  object
keywords                 object
poster_path              object
backdrop_path            object
recommendations          object
OverviewVector           object
dtype: object


In [39]:
from elasticsearch import Elasticsearch, TransportError, ConnectionError, RequestError

# Create the Elasticsearch client with authentication
es = Elasticsearch(
    hosts=["http://localhost:9200"],
    http_auth=("elastic", "DwOAtXZC3kMLCPadyoQY"),  
    request_timeout=120
)
index_body = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym_filter": {
                    "type": "synonym",
                    "synonyms_path": "synonyms.txt"  
                }
            },
            "analyzer": {
                "synonym_analyzer": {
                    "tokenizer": "whitespace",
                    "filter": ["lowercase", "synonym_filter"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "id": {"type": "long"},
            "title": {"type": "text", "analyzer": "standard"},
            "genres": {"type": "text"},
            "original_language": {"type": "text"},
            "overview": {"type": "text", "analyzer": "standard"},
            "popularity": {"type": "long"},
            "production_companies": {"type": "text"},
            "release_date": {"type": "date", "format": "yyyy-MM-dd"},
            "budget": {"type": "long"},
            "revenue": {"type": "long"},
            "runtime": {"type": "long"},
            "status": {"type": "text"},
            "tagline": {"type": "text", "analyzer": "standard"},
            "vote_average": {"type": "float"},
            "vote_count": {"type": "long"},
            "credits": {"type": "text", "analyzer": "standard"},
            "keywords": {"type": "text"},
            "poster_path": {"type": "text"},
            "backdrop_path": {"type": "text"},
            "recommendations": {"type": "text", "analyzer": "standard"},
            "OverviewVector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "l2_norm"
            }
        }
    }
}

# Check if the index already exists
if es.indices.exists(index="all_movies"):
    print("Index exists, deleting...")
    es.indices.delete(index="all_movies")

# Create the index with both settings and mappings
try:
    es.indices.create(index="all_movies", body=index_body)
    print("Index created successfully.")
except RequestError as e:
    print(f"RequestError: {e.info}")
except TransportError as e:
    print(f"TransportError: {e.info}")
except ConnectionError as e:
    print(f"ConnectionError: {str(e)}")
except Exception as e:
    print(f"Unexpected error: {e}")
    





C:\Users\sajan\AppData\Local\Temp\ipykernel_25692\3785875669.py:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Index exists, deleting...
Index created successfully.


In [40]:
record_list = df.to_dict("records")

In [50]:
for record in record_list:
    try:
        es.index(index="all_movies", document=record, id=record["id"])
    except Exception as e:
        print(e)

BadRequestError(400, 'document_parsing_exception', "[1:329] failed to parse field [release_date] of type [date] in document with id '616851'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:822] failed to parse field [release_date] of type [date] in document with id '496276'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:329] failed to parse field [release_date] of type [date] in document with id '616851'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:257] failed to parse field [release_date] of type [date] in document with id '730629'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:312] failed to parse field [release_date] of type [date] in document with id '852845'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:349] failed to parse field [release_date] of type [date] in 

In [51]:
es.count(index="all_movies")

ObjectApiResponse({'count': 46042, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [52]:
# Use the synonym analyzer on input keyword
input_keyword = "Super Hero"
analyzed_keyword = es.indices.analyze(index="all_movies", body={
    "analyzer": "synonym_analyzer",
    "text": input_keyword
})


processed_keyword = " ".join([token["token"] for token in analyzed_keyword["tokens"]])


vector_of_input_keyword = model.encode(processed_keyword)

# Perform KNN search
query = {
    "field": "OverviewVector",
    "query_vector": vector_of_input_keyword,
    "k": 2,
    "num_candidates": 500,
}

# KNN search with vector and return relevant fields
res = es.knn_search(index="all_movies", knn=query, source=["title", "overview", "genres", "vote_average", "poster_path", "backdrop_path", "combined_overview"])

# Display hits
print(res["hits"]["hits"])

[{'_index': 'all_movies', '_id': '287590', '_score': 0.51970947, '_source': {'title': 'Saint Seiya: Legend of Sanctuary', 'genres': 'Action-Animation-Fantasy', 'overview': 'Warriors called "Saints" are the champions of hope who have always appeared since the Age of Myth whenever evil threatens the world. In this present day story many years since the long fought "Holy War" we find Saori Kido a girl troubled by her mysterious powers. She is saved by a boy Seiya "Bronze Saint" from a sudden attack by an assassin through the accident Saori realizes her destiny and mission and decides to go to "Sanctuary" with Seiya and his company of Bronze Saints. In Sanctuary they confront "Pope" and wage a desperate battle against the greatest Saints the "Gold Saints".', 'vote_average': 6.5, 'poster_path': '/gxKfP9278RK4t3J8vLXoFfB8ihl.jpg', 'backdrop_path': '/odBSDNyi5J6Sed2Xu3QKTtEAIzN.jpg'}}, {'_index': 'all_movies', '_id': '178592', '_score': 0.5085058, '_source': {'title': 'Tiger & Bunny: The Begi

C:\Users\sajan\AppData\Local\Temp\ipykernel_25692\3940546205.py:23: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_movies", knn=query, source=["title", "overview", "genres", "vote_average", "poster_path", "backdrop_path", "combined_overview"])


In [53]:
def search_movies(query):
  # search movies using the Elasticsearch `search` method
  # and return the search results
  res = es.search(index="all_movies", body=query)
  return res
def filter_movies(title=None, genres=None):
  # create a bool query to filter the movies
  bool_query = {
    "bool": {
      "must": []
    }
  }
  # add filters to the bool query based on the provided parameters
  if title:
    bool_query["bool"]["must"].append({"match": {"name": title}})
  if genres:
    bool_query["bool"]["must"].append({"match": {"genre": genres}})
  
  # create the Elasticsearch query
  query = {
    "query": bool_query
  }
  # search and return the movies
  return search_movies(query)

In [54]:
# Example of multiple queries and their relevant results in the movie domain
queries = {
     
   
    "A detective solving a mystery in a small town": ["Sherlock Holmes", "Knives Out", "Jar City", "Uncle John"],
    "a panda performing martial arts": ["Kung Fu Panda", "Kung Fu Panda 2", "Kung Fu Panda Holiday", "Kung Fu Panda 3"]
}

# Example function to calculate Precision@K
def precision_at_k(k, retrieved, relevant):
    retrieved_k = retrieved[:k]
    relevant_k = [item for item in retrieved_k if item in relevant]
    precision = len(relevant_k) / k
    return precision

# Function to calculate Average Precision (AP)
def average_precision(retrieved, relevant):
    num_relevant = 0
    score_sum = 0
    for k, doc in enumerate(retrieved, start=1):
        if doc in relevant:
            num_relevant += 1
            score_sum += precision_at_k(k, retrieved, relevant)
    
    if num_relevant == 0:
        return 0.0
    return score_sum / num_relevant

# Function to calculate Mean Average Precision (MAP)
def mean_average_precision(queries, retrieved_results):
    ap_sum = 0
    for query, relevant_docs in queries.items():
        retrieved_docs = retrieved_results.get(query, [])
        ap = average_precision(retrieved_docs, relevant_docs)
        ap_sum += ap
        print(f"AP for query '{query}': {ap}")
    
    map_score = ap_sum / len(queries)
    return map_score

# Simulating search results for each query with 4 retrieved movies
retrieved_results = {
   
    
    "A detective solving a mystery in a small town": ["Jar City", "Winterkartoffelknödel", "Reptile", "Uncle John"],
    "a panda performing martial arts": ["Pandas", "Kung Fu Panda", "Kung Fu Panda 2", "Turning Red"]
}

# Calculate Mean Average Precision (MAP)
map_score = mean_average_precision(queries, retrieved_results)
print(f"Mean Average Precision (MAP): {map_score}")


AP for query 'A detective solving a mystery in a small town': 0.75
AP for query 'a panda performing martial arts': 0.5833333333333333
Mean Average Precision (MAP): 0.6666666666666666


: 